# Respiration

In [ ]:
def rr(start, end, patient_id, print1, print2):
    if patient_id == 'simulator':
        time = pickle.load(open('samples/simulated_time', 'rb'))
        rr = pickle.load(open('samples/simulated_processed_rr', 'rb'))
        return time[(int)(start*F):(int)(end*F)], rr[(int)(start*F):(int)(end*F)]
    
    debug = print1 or print2
    if debug == False and os.path.exists('pickles/'+str(patient_id)+'-rr-processed') == True:
        time = pickle.load(open('pickles/'+str(patient_id)+'-time-processed', 'rb'))
        rr = pickle.load(open('pickles/'+str(patient_id)+'-rr-processed', 'rb'))
        return time[(int)(start*F):(int)(end*F)], rr[(int)(start*F):(int)(end*F)]
    
    if debug == False:
        start_ = 0
        end_ = 2300
    else:
        start_ = start
        if start != 0:
            start_ = start - MARGIN
        end_ = end + MARGIN
    resp = []
    filename = patient_id + "-MDC_RESP-62.5.csv"
    if print1 or print2:
        print(filename+':')
        print("sampling frequency is ", F," Hz")
    with open('samples/'+filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            resp.append(float(row[1]))

    resp_small = np.array(resp[(int)(start_*62.5):(int)(end_*62.5)])            
    peaks,_=find_peaks(resp_small, distance=31, height=1150, prominence=1)
    peaks_sec = (peaks + start_*62.5) * 0.016
    time = np.arange(start_, end_, 0.016)

    if print1:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(time,resp_small)
        plt.plot(peaks_sec, resp_small[peaks], "x")
        plt.title('Respiration')
        plt.xlabel('Time [sec]')
        plt.show()

    diffs = np.diff(peaks_sec)
    peaks_sec = peaks_sec[:len(diffs)]+diffs/2
    rr = 60/diffs

    rr_interp_func = interp1d(peaks_sec, rr)
    time = np.arange(start_ + MARGIN, end_ - MARGIN + 1/F, 1/F)

    if print2:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(peaks_sec,rr)
        plt.plot(time,rr_interp_func(time))
        plt.title('Respiration Rate')
        plt.xlabel('Time [sec]')
        plt.ylabel('[Hz]')
        plt.legend(['data', 'resampled'], loc='best')
        plt.show()
    
    if debug == False:
        with open('pickles/'+str(patient_id)+'-rr-processed', 'wb') as rr_file:
            pickle.dump(rr_interp_func(time), rr_file)
        return time[(int)(start*F):(int)(end*F)], rr[(int)(start*F):(int)(end*F)]
